In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html

import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#Python Module and class name
from CRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# username and password and CRUD Python module name

username = "aacuser"
password = "Smilax25"

# Connect to database via CRUD Module
db = AnimalShelter()

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
#

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' #png image file
encoded_image = base64.b64encode(open(image_filename, 'rb').read())



#design code
app.layout = html.Div([
    html.A([html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                    height =250, width = 251))]),
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Animal Shelter'))),
    html.Center(html.I(html.H3('Creator: Shawn Millin'))), #unique identifier
    html.Hr(),
    html.H3('Rescue Type'),
    html.Div(dcc.RadioItems(id='radio-filter',
                            options=[
                                {'label':'Reset','value':'reset'},
                                {'label':'Water','value':'water'},
                                {'label':'Mountain or Wilderness','value':'wild'},
                                {'label':'Disaster or Individual','value':'disaster'}
                            ],
                            value='reset' #initialize table with only prefered dog breeds
                           )),
                   
    
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True}
                                  for i in df.columns],
                         data=df.to_dict('records'),
                        #interactive data table to make it user-friendly for clients
                            editable=False,
                            sort_action = "native",  #sort column  
                            row_selectable = "single",  #select a single row
                            row_deletable = False,  #not able to delete row
                            page_action="native",
                            selected_rows=[0],
                            page_current=0,
                            page_size = 12,  #12 entries per page

                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
############################################# 
   
@app.callback([Output('datatable-id','data'),
              Output('datatable-id','columns')],
              [Input('radio-filter', 'value')])
def on_click(radio_type):
##code to filter interactive data table with MongoDB queries
     
    #Reset shows only the Prefered Dogs for Grazioso Salvre
    if radio_type == 'reset':  
        df = pd.DataFrame.from_records(db.read({
           "breed":{'$in':["German Shepherd","Chesapeake Bay Retriever", "Newfoundland",
            "Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler",
            "Doberman Pinscher","Bloodhound","Golden Retriever"]}}))
    #each of the following are specified criteria for each rescue type    
    elif radio_type == 'water':        
        df = pd.DataFrame.from_records(db.read({
            'breed':{'$in':["Labrador Retriever Mix",
            "Chesapeake Bay Retriever", "Newfoundland"]},
            'sex_upon_outcome':"Intact Female",
            'age_upon_outcome_in_weeks':{'$gt':26.0,'$lt':156.0}
        }))
   
    elif radio_type == 'wild':
        df = pd.DataFrame.from_records(db.read({
            "breed":{'in':["German Shepherd",
            "Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]},
            'sex_upon_outcome':"Intact Male",
            'age_upon_outcome_in_weeks':{'$gt':26.0,'$lt':156.0}
        }))
        
    elif  radio_type == 'disaster':
        df = pd.DataFrame.from_records(db.read({
            "breed":{'$in':["Doberman Pinscher",
            "Golden Retriever","German Shepherd","Bloodhound","Rottweiler"]},
            "sex_upon_outcome":"Intact Male", 
            'age_upon_outcome_in_weeks':{'$gt':20.0,'$lt':300.0}
        }))

   
  
         
        
    #df.drop(columns=['_id'],inplace=True)
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
   
      
#       
    return (data,columns)

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
   
    # code for pie chart 
    dff= pd.DataFrame.from_dict(viewData)
    return [
       dcc.Graph( 
           
           figure = px.pie(dff, names='breed', title='Preferred Animals')
      )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)
